# **DESCRIPTA.AI** -> LLM Service Wrapper (llm_service.py)

Lightweight client that wraps the official Ollama SDK, forwarding all calls through your `OLLAMA_HOST` (or `LLM_MODEL`) and preserving features like streaming and automatic retries.

## File: `llm_service.py`

In [ ]:
import os
import logging
import ollama
from typing import List, Optional

class LLMService:

    def __init__(self, model: Optional[str] = None):
        # Toma el modelo de env var LLM_MODEL o usa "llama4:maverick" por defecto
        self.model = model or os.getenv("LLM_MODEL", "llama4:maverick")

    def is_available(self, timeout: float = 3.0) -> bool:

        try:
            ollama.show(model=self.model)
            return True
        except Exception as e:
            logging.warning(f"LLM remoto no disponible: {e}")
            return False

    def chat(self, prompt: str, images: Optional[List[str]] = None) -> str:

        message = {"role": "user", "content": prompt}
        if images:
            message["images"] = images

        try:
            response = ollama.chat(
                model=self.model,
                messages=[message]
            )
            return response.message.content.strip()
        except Exception as e:
            logging.error(f"Error en LLMService.chat(): {e}")
            return ""

## Usage Example

In [ ]:
from llm_service import LLMService

# Initialize (reads LLM_MODEL env var or defaults to "llama4:maverick")
llm = LLMService()

# Health check
if llm.is_available():
    answer = llm.chat("Hello, Ollama!")
    print("Model replied:", answer)
else:
    print("LLM service is not reachable.")


## Public API

* LLMService(model: Optional[str])

Create a client for the given model name (or use LLM_MODEL env var).

* is_available(timeout: float = 3.0) → bool

Returns whether the remote model server is up (via ollama.show).

* chat(prompt: str, images: Optional[List[str]] = None) → str

Send a text (and optional images) to the model; returns the trimmed reply or "" on error.